# Analyze and Visualize Results

In [1]:
import logging
import json
from pathlib import Path
from typing import List, Tuple, Union, Dict, Optional
import sys
from multiprocessing import Process, Pool
import psutil
import base64
import asyncio
import cv2
import imageio as iio
import numpy as np
from IPython.core.display import HTML
from matplotlib import pyplot as plt
from scipy.fft import fft, fftfreq, fftn, fftshift

from turbx import REPO_PATH, log
from turbx.vis import (calc_tfpnr, calc_tdr, calc_frames_removed, label_to_per_frame_list)
from turbx.metrics import (
    calc_box_area,
    boxes_to_binary,
    target_detection_rate,
    tfpnr,
)

[DEBUG] 10/28/2022 01:42:15PM: MainProcess: __init__.py - Loaded logging config file: /Users/nowa201/Code/fish_detector/turbx/src/turbx/logging.yaml


### Load experiment output

In [9]:
base_path = f"{REPO_PATH}/experiments/dft/outputs/"
#exp_path = "2022-10-27/10-57-06/0"  # no tracklet - best
#exp_path = "2022-10-27/13-06-28/0"  # tracklet low fpr - best FPR
#exp_path = "2022-10-27/13-35-41/0"   # tracklet low fnr
exp_path = "2022-10-28/13-48-14/0"  # tracklet w/ params as no tracklet
exp_path = Path(base_path + exp_path)

all_results = []
for f_name in Path(exp_path).glob("**/*.json"):
    with open(f_name, 'r') as f:
        params = json.load(f)
        all_results.append(params)
        

### Calculate metrics on experiment data

In [10]:
# calculate per video scores


all_neg_dets= 0
all_neg_lab= 0
all_frames =0
print("Per frame:")
print("id:, FPR, FNR, Target Detection Rate, % Frames Removed, % Neg Frames Removed")
for result in all_results:
    binary_label, binary_pred, tfpnr_dict = calc_tfpnr(result['label'], result['prediction'], show=False, save=False)
    unique_targs, det_targs, tdr = calc_tdr(result['label'], result['prediction'])
    _, n_pos_dets, n_neg_dets = calc_frames_removed(result['prediction'])
    _, n_pos_lab, n_neg_lab = calc_frames_removed(label_to_per_frame_list(result['label']))
    perc_frames_removed = 100*(n_neg_dets / len(binary_pred))
    perc_neg_frame_removed = 100*(n_neg_dets / n_neg_lab)
    all_neg_dets += n_neg_dets
    all_neg_lab += n_neg_lab
    all_frames += len(binary_pred)
    print(f"\n{result['label']['video_id']}: {tfpnr_dict['fpr']:.2f}, {tfpnr_dict['fnr']:.2f}, {tdr:.2f}, {perc_frames_removed:.2f}, {perc_neg_frame_removed:.2f}")
perc_all_frames = 100*(all_neg_dets/all_frames)
perc_tn_frames = 100*(all_neg_lab/all_frames)
print(f"\n% of all frames removed: {perc_all_frames:.2f}")
print(f"\n% of all frames that are empty: {perc_tn_frames:.2f}")

Per frame:
id:, FPR, FNR, Target Detection Rate, % Frames Removed, % Neg Frames Removed

32: 0.00, 0.60, 1.00, 94.87, 108.82

34: 0.18, 0.24, 1.00, 67.22, 90.30

33: 0.09, 0.14, 1.00, 81.14, 92.96

20: 0.00, 1.00, 0.00, 100.00, 136.84

18: 0.44, 0.26, 1.00, 50.24, 61.63

27: 0.68, 0.17, 0.93, 29.44, 36.51

29: 0.37, 0.00, 1.00, 61.03, 63.41

16: 0.76, 0.00, 1.00, 21.51, 24.39

17: 0.70, 0.00, 1.00, 24.24, 29.63

10: 0.83, 0.05, 1.00, 10.24, 24.07

19: 0.03, 1.00, 0.00, 97.62, 110.81

26: 0.02, 0.90, 0.11, 96.81, 109.46

21: 0.00, 0.79, 0.67, 97.41, 110.78

31: 0.14, 0.31, 1.00, 80.43, 89.97

30: 0.39, 0.13, 0.83, 53.75, 63.55

23: 0.13, 1.00, 0.00, 89.20, 104.89

15: 0.73, 0.00, 1.00, 26.67, 26.67

13: 0.88, 0.00, 1.00, 10.00, 12.12

14: 0.48, 0.00, 1.00, 48.44, 51.66

22: 0.02, 1.00, 0.00, 98.74, 124.60

25: 0.00, 0.32, 0.92, 84.55, 109.41

% of all frames removed: 61.11

% of all frames that are empty: 83.94
